In [ ]:
from pathlib import Path
import pandas as pd
import os, json, glob, time, subprocess, shlex, pathlib
start = 0
SLEEP_BETWEEN = 0

AUTH_USER="samuelzhu"
AUTH_PASS="Xdb73008762"

TOKEN_URL = "https://analysis.yamaguchi.lan/AppleStockChecker/auth/token/"
TOKEN_TIMEOUT = 90
CERT_BUNDLE = "../caddy-ca-bundle.crt"

URL = "https://analysis.yamaguchi.lan/AppleStockChecker/purchasing-price-records/import-tradein-xlsx/?dry_run=0&upsert=1"

RETRIES = 1
BACKOFF = 2.0
TMP_DIR = Path(".tmp_csv_uploads")                # 转换后的临时 CSV 存放目录
TMP_DIR.mkdir(exist_ok=True)
PATTERNS = ["**/*.xlsx", "**/*.xls", "**/*.xlsm", "**/*.xlsb", "**/*.ods", "**/*.csv"]
ENGINE_HINT = {
    ".xlsx": "openpyxl",
    ".xlsm": "openpyxl",
    ".xls":  "xlrd",    # 需要 xlrd<2.0
    ".xlsb": "pyxlsb",
    ".ods":  "odf",
}


def fetch_access_token(token_url: str = TOKEN_URL) -> str:
  """
  获取JWT访问令牌（使用curl实现）

  保持原有接口不变，内部使用subprocess调用curl
  """
  auth_bearer = os.getenv("AUTH_BEARER", "").strip()
  auth_user = AUTH_USER
  auth_pass = AUTH_PASS

  headers = ["Accept: application/json"]
  payload = None

  if auth_bearer:
      headers.append(f"Authorization: Bearer {auth_bearer}")
      payload = {}
  elif auth_user and auth_pass:
      payload = {"username": auth_user, "password": auth_pass}
  else:
      raise RuntimeError(
          "未提供 ACCESS，且缺少获取 token 所需的凭据。\n"
          "请设置以下之一：\n"
          "  1) export ACCESS=...  (直接跳过自动获取)\n"
          "  2) export AUTH_BEARER=... (用 Bearer 访问 token 接口)\n"
          "  3) export AUTH_USER=... && export AUTH_PASS=... (用户名密码换 token)\n"
      )

  # 构建 curl 命令
  curl_command = [
      'curl',
      '--cacert', CERT_BUNDLE,
      '-X', 'POST',
      '--silent',
      '--show-error',
      '--fail',
      '--max-time', str(TOKEN_TIMEOUT),
      '-w', '\n%{http_code}',
  ]

  # 添加请求头
  for header in headers:
      curl_command.extend(['-H', header])

  # 添加请求体
  if payload is not None:
      curl_command.extend([
          '-H', 'Content-Type: application/json',
          '-d', json.dumps(payload)
      ])

  curl_command.append(token_url)

  # 执行 curl
  try:
      result = subprocess.run(
          curl_command,
          capture_output=True,
          text=True,
          timeout=TOKEN_TIMEOUT + 5,
          check=False
      )

      # 分离响应体和状态码
      output_lines = result.stdout.strip().split('\n')
      if len(output_lines) >= 2:
          response_body = '\n'.join(output_lines[:-1])
          http_code = output_lines[-1]
      else:
          response_body = result.stdout.strip()
          http_code = "000"

      # 检查返回码
      if result.returncode != 0:
          error_msg = result.stderr.strip() or "curl命令执行失败"
          raise RuntimeError(
              f"请求 token 接口失败：{token_url}\n"
              f"curl 返回码: {result.returncode}\n"
              f"错误: {error_msg}\n"
              f"响应: {response_body[:500]}"
          )

      # 检查 HTTP 状态码
      try:
          status_code = int(http_code)
      except ValueError:
          status_code = 0

      if status_code < 200 or status_code >= 300:
          raise RuntimeError(
              f"获取 token 失败：HTTP {status_code}\n"
              f"URL: {token_url}\n"
              f"Response: {response_body[:800]}"
          )

      # 解析响应
      try:
          data = json.loads(response_body)
      except json.JSONDecodeError:
          token = response_body.strip()
          if token:
              return token
          raise RuntimeError(f"token 接口返回非 JSON 且为空：{response_body[:200]}")

      # 兼容常见字段名
      for k in ("access", "token", "access_token", "key"):
          v = data.get(k)
          if isinstance(v, str) and v.strip():
              return v.strip()

      raise RuntimeError(
          f"token 接口返回 JSON 但未找到 token 字段：\n"
          f"keys={list(data.keys())}, data={data}"
      )

  except subprocess.TimeoutExpired:
      raise RuntimeError(f"请求 token 接口超时：{token_url} ({TOKEN_TIMEOUT}秒)")
  except FileNotFoundError:
      raise RuntimeError("curl 命令未找到，请确保系统已安装 curl")
  except Exception as e:
      if isinstance(e, RuntimeError):
          raise
      raise RuntimeError(f"请求 token 接口失败：{token_url} ; err={e}") from e

def resolve_access_token() -> str:
    """
    统一入口：优先使用环境变量 ACCESS；否则自动获取。
    """
    return fetch_access_token()

def list_files(root: str | Path) -> list[str]:
    files: list[str] = []
    for pat in PATTERNS:
        files += glob.glob(str(Path(root) / pat), recursive=True)
    # 过滤临时文件 ~$
    return sorted(f for f in files if not Path(f).name.startswith("~$"))

def to_csv_if_needed(path: str | Path) -> Path:
    """
    若是 Excel -> 转 CSV 到 TMP_DIR；若已是 CSV 直接返回原路径。
    返回：CSV 文件路径
    """
    p = Path(path)
    suf = p.suffix.lower()
    if suf == ".csv":
        return p

    engine = ENGINE_HINT.get(suf)
    # 读 Excel 为 DataFrame（默认取第一个 sheet；如需更复杂可扩展）
    try:
        df = pd.read_excel(p, engine=engine) if engine else pd.read_excel(p)
    except ImportError as e:
        raise RuntimeError(f"缺少读取 {suf} 的依赖（{e}）。请按需安装：openpyxl / xlrd<2 / pyxlsb / odfpy") from e
    except Exception as e:
        raise RuntimeError(f"读取 {p.name} 失败：{e}") from e

    # 写到临时 CSV（UTF-8 BOM，便于日后人工打开）
    out = TMP_DIR / (p.parent.name + ".csv")
    try:
        df.to_csv(out, index=False, encoding="utf-8-sig")
    except Exception as e:
        raise RuntimeError(f"写出临时 CSV 失败：{out.name}，{e}") from e
    return out

def post_one(csv_path: Path, access: str, retries: int = 2, backoff: float = 2.0) -> tuple[str, str]:
    """
    逐个文件 POST；返回 (状态行, 整个响应字符串)
    """
    # 用 curl 发送 multipart；注意不要把 -H 里的 token 打印到日志
    cmd = [
        "curl", "-sS", "-i", "-X", "POST", URL,'--cacert', CERT_BUNDLE,
        "-H", f"Authorization: Bearer {access}",
        "-F", f"files=@{str(csv_path)}",
    ]
    print(f"POST {csv_path}")
    last_err = ""
    for attempt in range(retries + 1):
        try:
            out = subprocess.check_output(cmd, stderr=subprocess.STDOUT)
            text = out.decode("utf-8", "replace")
            status_line = text.splitlines()[0] if text else ""
            return status_line, text
        except subprocess.CalledProcessError as e:
            last_err = e.output.decode("utf-8", "replace")
            if attempt >= retries:
                return f"ERROR after {retries} retries", last_err
            time.sleep(backoff * (attempt + 1))
    return "ERROR", last_err

def curl_request(url, method='GET', headers=None, json_data=None, timeout=30):
  """通用curl请求封装"""
  curl_command = [
      'curl',
      '--cacert', CERT_BUNDLE,
      '-X', method.upper(),
      '--silent',
      '--show-error',
      '--max-time', str(timeout),
      '-w', '\n%{http_code}',
  ]

  if headers:
      for key, value in headers.items():
          curl_command.extend(['-H', f'{key}: {value}'])

  if json_data:
      curl_command.extend([
          '-H', 'Content-Type: application/json',
          '-d', json.dumps(json_data)
      ])

  curl_command.append(url)

  result = subprocess.run(curl_command, capture_output=True, text=True, timeout=timeout+5)
  output_lines = result.stdout.strip().split('\n')
  response_body = '\n'.join(output_lines[:-1])
  http_code = int(output_lines[-1])

  return http_code, response_body

def main(ROOT):
    files = list_files(ROOT)
    print(f"Found {len(files)} files under {ROOT}")

    success = fail = 0
    for i, f in enumerate(files[start:], 1):
        try:
            ACCESS = resolve_access_token()
        except Exception as e:
            raise SystemExit(f"无法获得 ACCESS：{e}")
        src = Path(f)
        print(src)
        # 从文件名提取清洗器名：<dir>/<shopX>.<ext> → shopX
        source_name = src.parent.name
        print(f"[{i}/{len(files)}] {src}  ->  source={source_name}")

        try:
            csv_path = to_csv_if_needed(src)
        except Exception as e:
            fail += 1
            print(f"  ✗ 转换失败：{e}")
            continue
        print("准备发送")
        status, resp = post_one(csv_path, access=ACCESS, retries=RETRIES, backoff=BACKOFF)
        print("  ", status)
        # 简要解析是否 202/200
        ok = status.startswith("HTTP/1.1 202") or status.startswith("HTTP/2 202") or \
             status.startswith("HTTP/1.1 200") or status.startswith("HTTP/2 200")
        if not ok:
            fail += 1
            # 打印一小段响应帮助定位
            snippet = "\n".join(resp.splitlines()[:15])
            print(snippet)
        else:
            success += 1
        time.sleep(SLEEP_BETWEEN)
    print(f"\nDone. success={success}, fail={fail}, tmp_csv_dir={TMP_DIR.resolve()}")
    # 如需清理临时 CSV，解除注释：
    # shutil.rmtree(TMP_DIR, ignore_errors=True)


ROOT = "../shop-data/shop3/"
main(ROOT)

Found 7706 files under ../shop-data/shop16/
../shop-data/shop16/00035450548-2025-10-30-shop16.xlsx
[1/7706] ../shop-data/shop16/00035450548-2025-10-30-shop16.xlsx  ->  source=shop16
准备发送
POST .tmp_csv_uploads/shop16.csv
   HTTP/2 202 
../shop-data/shop16/00035450637-2025-10-30-shop16.xlsx
[2/7706] ../shop-data/shop16/00035450637-2025-10-30-shop16.xlsx  ->  source=shop16
准备发送
POST .tmp_csv_uploads/shop16.csv
   HTTP/2 202 
../shop-data/shop16/00035451655-2025-10-30-shop16.xlsx
[3/7706] ../shop-data/shop16/00035451655-2025-10-30-shop16.xlsx  ->  source=shop16
准备发送
POST .tmp_csv_uploads/shop16.csv
   HTTP/2 202 
../shop-data/shop16/00035451908-2025-10-30-shop16.xlsx
[4/7706] ../shop-data/shop16/00035451908-2025-10-30-shop16.xlsx  ->  source=shop16
准备发送
POST .tmp_csv_uploads/shop16.csv
   HTTP/2 202 
../shop-data/shop16/00035452286-2025-10-30-shop16.xlsx
[5/7706] ../shop-data/shop16/00035452286-2025-10-30-shop16.xlsx  ->  source=shop16
准备发送
POST .tmp_csv_uploads/shop16.csv
   HTTP/2 202 
.

KeyboardInterrupt: 